# Birmingham H₂ Bus Fleet – Techno-Economic Analysis

**Author:** Inigo Antony Michael Selvam  
**Programme:** MSc Sustainable Energy Systems, University of Birmingham / IIT Madras  
**Course Module:** 36869 Fuel Cell Electric Vehicles 2024–2025  

---

This notebook implements and extends the analysis from the coursework report:  
*"Analysis of Hydrogen Refuelling Infrastructure for Birmingham's Fuel Cell Bus Fleet"*

**Scope:**
1. Hydrogen demand modelling for 140-bus FCEV fleet
2. Infrastructure CAPEX (12 MWe electrolyser + 3 HRS stations)
3. Levelised Cost of Hydrogen (LCOH) at the dispenser
4. Well-to-Wheel CO₂ emissions comparison vs diesel
5. **Sensitivity analysis** – LCOH vs electricity price
6. **Breakeven analysis** – diesel price parity
7. **NPV / IRR** – infrastructure investment with carbon cost benefits

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import sys
import os

# Ensure project root is on path
sys.path.insert(0, os.path.abspath('.'))

# Project modules
from src.parameters import *
from src.demand import calculate_demand
from src.infrastructure import calculate_capex
from src.economics import calculate_lcoh, calculate_annual_costs, calculate_npv_irr, diesel_breakeven_price
from src.emissions import calculate_emissions

plt.rcParams.update({
    'figure.dpi': 130, 'figure.facecolor': 'white',
    'axes.facecolor': '#F8F9FA', 'axes.grid': True,
    'grid.color': '#DDDDDD', 'grid.linewidth': 0.7,
    'axes.spines.top': False, 'axes.spines.right': False,
    'font.size': 11, 'axes.titleweight': 'bold',
})

PALETTE = {
    'blue': '#1A6CA8', 'orange': '#E07B39', 'green': '#2E8B57',
    'red': '#C0392B', 'purple': '#7D3C98', 'grey': '#7F8C8D',
}
print('✓ Modules loaded successfully')

---
## 1. Hydrogen Demand

In [ ]:
d = calculate_demand()

print(f"{'Parameter':<40} {'Value':>15}")
print("-" * 57)
print(f"{'Fleet size':<40} {'140 buses':>15}")
print(f"{'Fuel economy':<40} {f'{FUEL_ECONOMY_KG_100KM} kg/100 km':>15}")
print(f"{'Daily mileage per bus':<40} {f'{DAILY_MILEAGE_KM} km':>15}")
print(f"{'Daily H₂ per bus':<40} {f'{d.daily_per_bus_kg:.1f} kg':>15}")
print(f"{'Full fleet daily demand':<40} {f'{d.full_fleet_daily_kg:.0f} kg/day':>15}")
print(f"{'Existing Tyseley capacity':<40} {f'{EXISTING_TYSELEY_CAPACITY_KG_DAY:,} kg/day':>15}")
print(f"{'Supply gap':<40} {f'{d.supply_gap_kg_day:.0f} kg/day':>15}")
print(f"{'Annual demand':<40} {f'{d.annual_total_tonnes:.0f} t/yr':>15}")

---
## 2. Infrastructure CAPEX

In [ ]:
cap = calculate_capex()

fig, ax = plt.subplots(figsize=(8, 4.5))
items = ['Electrolyser\nequipment', 'BoP &\ninstallation', 'HRS network\n(3 stations)']
values = [cap.electrolyser_equipment_gbp/1e6, cap.electrolyser_bop_gbp/1e6, cap.hrs_total_gbp/1e6]
colours = [PALETTE['blue'], '#AED6F1', PALETTE['orange']]
bars = ax.bar(items, values, color=colours, width=0.45, edgecolor='white', zorder=3)
ax.bar_label(bars, fmt='£%.2fM', padding=5, fontsize=11, fontweight='bold')
ax.axhline(cap.total_capex_gbp/1e6, color=PALETTE['red'], ls='--', lw=1.5)
ax.text(2.35, cap.total_capex_gbp/1e6 + 0.1, f"Total: £{cap.total_capex_gbp/1e6:.2f}M",
        color=PALETTE['red'], ha='right', fontweight='bold')
ax.set_ylabel('Capital Cost (£M)')
ax.set_title('Infrastructure CAPEX Breakdown')
ax.set_ylim(0, cap.total_capex_gbp/1e6 * 1.2)
ax.grid(axis='x', visible=False)
plt.tight_layout()
plt.show()

print(f"\nTotal CAPEX: £{cap.total_capex_gbp/1e6:.2f}M  "
      f"(Electrolyser: £{cap.electrolyser_total_gbp/1e6:.2f}M | HRS: £{cap.hrs_total_gbp/1e6:.2f}M)")

---
## 3. LCOH Breakdown at Baseline

In [ ]:
lc = calculate_lcoh(ELECTRICITY_PRICE_GBP_MWH)

components = {
    'Electricity': lc.electricity_cost_gbp_kg,
    'CAPEX (amort.)': lc.capex_amortised_gbp_kg,
    'Non-energy OPEX': lc.opex_non_energy_gbp_kg,
    'Stack replacement': lc.stack_replacement_gbp_kg,
    'Transport (GH₂)': lc.transport_gbp_kg,
    'HRS operations': lc.hrs_opex_gbp_kg,
}

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 4.5))

# Pie
colours_pie = [PALETTE['blue'], PALETTE['orange'], PALETTE['green'],
               PALETTE['purple'], PALETTE['grey'], PALETTE['red']]
wedges, texts, autotexts = ax1.pie(
    list(components.values()),
    labels=list(components.keys()),
    colors=colours_pie,
    autopct='%1.1f%%', startangle=140, pctdistance=0.8,
    textprops={'fontsize': 9}
)
ax1.set_title(f'LCOH Components  (Total: £{lc.total_dispensed_cost_gbp_kg:.2f}/kg)')

# Horizontal bar
ax2.barh(list(components.keys()), list(components.values()),
         color=colours_pie, height=0.55, zorder=3)
for i, v in enumerate(components.values()):
    ax2.text(v + 0.02, i, f'£{v:.3f}', va='center', fontsize=10)
ax2.set_xlabel('£/kg H₂')
ax2.set_title('LCOH Component Values')
ax2.grid(axis='y', visible=False)

plt.suptitle(f'Baseline: £{ELECTRICITY_PRICE_GBP_MWH}/MWh electricity', fontsize=11, y=1.01)
plt.tight_layout()
plt.show()

---
## 4. Sensitivity Analysis – LCOH vs Electricity Price

In [ ]:
elec_range = np.linspace(20, 120, 80)

elec_costs, capex_costs, opex_costs, stack_costs, trans_costs, hrs_costs, totals = (
    [], [], [], [], [], [], []
)
for ep in elec_range:
    lc = calculate_lcoh(ep)
    elec_costs.append(lc.electricity_cost_gbp_kg)
    capex_costs.append(lc.capex_amortised_gbp_kg)
    opex_costs.append(lc.opex_non_energy_gbp_kg)
    stack_costs.append(lc.stack_replacement_gbp_kg)
    trans_costs.append(lc.transport_gbp_kg)
    hrs_costs.append(lc.hrs_opex_gbp_kg)
    totals.append(lc.total_dispensed_cost_gbp_kg)

fig, ax = plt.subplots(figsize=(9, 5.5))
bottom = np.zeros(len(elec_range))
layers = [
    ('Electricity', elec_costs, PALETTE['blue']),
    ('CAPEX (amort.)', capex_costs, PALETTE['orange']),
    ('Non-energy OPEX', opex_costs, PALETTE['green']),
    ('Stack replacement', stack_costs, PALETTE['purple']),
    ('Transport', trans_costs, PALETTE['grey']),
    ('HRS operations', hrs_costs, PALETTE['red']),
]
for label, values, col in layers:
    ax.bar(elec_range, values, bottom=bottom, width=1.1, label=label, color=col, alpha=0.85, linewidth=0)
    bottom += np.array(values)

ax.plot(elec_range, totals, color='black', lw=2.0, zorder=5, label='Total LCOH')
ax.axvline(ELECTRICITY_PRICE_GBP_MWH, color='black', ls='--', lw=1.2, alpha=0.6)
ax.annotate(f'Baseline\n£{ELECTRICITY_PRICE_GBP_MWH}/MWh',
            xy=(ELECTRICITY_PRICE_GBP_MWH, max(totals) * 0.85),
            fontsize=9, ha='center')

ax.set_xlabel('Electricity Price (£/MWh)')
ax.set_ylabel('Cost (£/kg H₂)')
ax.set_title('Levelised Cost of Hydrogen at Dispenser vs. Electricity Price')
ax.legend(loc='upper left', fontsize=9)
ax.set_xlim(20, 120)
ax.set_ylim(0, max(totals) * 1.12)
plt.tight_layout()
plt.show()

print(f"At £20/MWh:  £{calculate_lcoh(20).total_dispensed_cost_gbp_kg:.2f}/kg")
print(f"At £57/MWh:  £{calculate_lcoh(57).total_dispensed_cost_gbp_kg:.2f}/kg  (baseline)")
print(f"At £120/MWh: £{calculate_lcoh(120).total_dispensed_cost_gbp_kg:.2f}/kg")

---
## 5. Annual Fleet Fuel Cost: H₂ vs Diesel

In [ ]:
h2_costs, diesel_costs = [], []
for ep in elec_range:
    ac = calculate_annual_costs(ep)
    h2_costs.append(ac.h2_fuel_cost_gbp / 1e6)
    diesel_costs.append(ac.diesel_fuel_cost_gbp / 1e6)

h2_arr = np.array(h2_costs)
d_val = diesel_costs[0]

fig, ax = plt.subplots(figsize=(9, 5.5))
ax.plot(elec_range, h2_arr, color=PALETTE['blue'], lw=2.5, label='H₂ fleet total fuel cost')
ax.axhline(d_val, color=PALETTE['orange'], lw=2.5, ls='--',
           label=f'Diesel fleet (£{DIESEL_PRICE_GBP_LITRE}/L baseline)')
ax.fill_between(elec_range, h2_arr, d_val, where=(h2_arr < d_val),
                alpha=0.15, color=PALETTE['green'], label='H₂ cheaper')
ax.fill_between(elec_range, h2_arr, d_val, where=(h2_arr >= d_val),
                alpha=0.15, color=PALETTE['red'], label='Diesel cheaper')
ax.axvline(ELECTRICITY_PRICE_GBP_MWH, color='grey', ls=':', lw=1.3)

ax.set_xlabel('Electricity Price (£/MWh)')
ax.set_ylabel('Annual Fuel Cost (£M)')
ax.set_title('Annual Fleet Fuel Cost: Hydrogen vs. Diesel  (140-bus fleet)')
ax.legend(fontsize=9)
ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('£%.1fM'))
plt.tight_layout()
plt.show()

---
## 6. Breakeven Diesel Price

In [ ]:
carbon_scenarios = [0, 50, 100, 150]
colours_line = [PALETTE['blue'], PALETTE['green'], PALETTE['orange'], PALETTE['red']]

fig, ax = plt.subplots(figsize=(9, 5.5))
for cp, col in zip(carbon_scenarios, colours_line):
    bev = [diesel_breakeven_price(ep, cp) for ep in elec_range]
    ax.plot(elec_range, bev, lw=2.2, color=col, label=f'Carbon = £{cp}/t CO₂e')

ax.axhline(DIESEL_PRICE_GBP_LITRE, color='black', ls='--', lw=1.5, alpha=0.7,
           label=f'Current diesel £{DIESEL_PRICE_GBP_LITRE}/L')
ax.axhline(2.0, color='grey', ls=':', lw=1.2, alpha=0.6, label='Stressed diesel £2.00/L')
ax.axvline(ELECTRICITY_PRICE_GBP_MWH, color='grey', ls=':', lw=1.3, alpha=0.5)

ax.set_xlabel('Electricity Price (£/MWh)')
ax.set_ylabel('Breakeven Diesel Price (£/litre)')
ax.set_title('Diesel Breakeven Price for H₂ Fleet Cost Parity')
ax.legend(fontsize=9)
ax.set_xlim(20, 120)
ax.set_ylim(bottom=0)
plt.tight_layout()
plt.show()

# Print key breakeven table
print(f"{'Elec. price (£/MWh)':<22} {'BEV diesel @ £0/t (£/L)':<26} {'BEV diesel @ £50/t (£/L)':<26}")
print("-" * 74)
for ep in [20, 40, 57, 80, 100, 120]:
    b0  = diesel_breakeven_price(ep, 0)
    b50 = diesel_breakeven_price(ep, 50)
    print(f"{ep:<22} {b0:<26.2f} {b50:<26.2f}")

---
## 7. Well-to-Wheel Emissions

In [ ]:
em = calculate_emissions()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 5))

# Annual totals
cats = ['Diesel\nbaseline', 'H₂ fleet\n(offshore wind)']
vals = [em.diesel_annual_co2_tonnes, em.h2_annual_co2_tonnes]
bars = ax1.bar(cats, vals, color=[PALETTE['orange'], PALETTE['blue']], width=0.45, zorder=3)
ax1.bar_label(bars, fmt='%.0f t', padding=5, fontsize=11, fontweight='bold')
ax1.set_ylabel('Annual CO₂e Emissions (tonnes/yr)')
ax1.set_title('WtW Annual Emissions Comparison')
ax1.set_ylim(0, em.diesel_annual_co2_tonnes * 1.2)
ax1.text(0.5, em.diesel_annual_co2_tonnes * 0.55,
         f'−{em.co2_reduction_pct:.0f}%\n({em.co2_saving_tonnes_yr:,.0f} t CO₂e saved)',
         ha='center', color=PALETTE['green'], fontsize=11, fontweight='bold')
ax1.grid(axis='x', visible=False)

# H₂ breakdown
comps = ['Production\n(electrolysis)', 'HRS\n(compression)', 'Transport\n(truck)']
ef_vals = [em.production_co2_kg_kgh2, em.hrs_co2_kg_kgh2, em.transport_co2_kg_kgh2]
cols2 = [PALETTE['blue'], PALETTE['purple'], PALETTE['grey']]
bars2 = ax2.barh(comps, ef_vals, color=cols2, height=0.45, zorder=3)
ax2.bar_label(bars2, fmt='%.3f kg', padding=4, fontsize=10)
ax2.set_xlabel('Emission Factor (kg CO₂e / kg H₂)')
ax2.set_title('H₂ Pathway Emission Factor Breakdown')
ax2.set_xlim(0, max(ef_vals) * 1.35)
ax2.grid(axis='y', visible=False)

plt.tight_layout()
plt.show()

print(f"Total H₂ WtW EF: {em.total_h2_ef_kg_kgh2:.3f} kg CO₂e/kg H₂")
print(f"Annual saving:   {em.co2_saving_tonnes_yr:,.0f} tonnes CO₂e  ({em.co2_reduction_pct:.1f}% reduction)")

---
## 8. NPV Analysis

In [ ]:
carbon_scenarios = [0, 50, 100, 150, 200]
colours_npv = [PALETTE['grey'], PALETTE['blue'], PALETTE['green'], PALETTE['orange'], PALETTE['red']]

fig, ax = plt.subplots(figsize=(9, 5.5))
for cp, col in zip(carbon_scenarios, colours_npv):
    npvs = [calculate_npv_irr(ep, cp).npv_gbp / 1e6 for ep in elec_range]
    ax.plot(elec_range, npvs, lw=2.2, color=col, label=f'Carbon = £{cp}/t CO₂e')

ax.axhline(0, color='black', lw=1.5)
ax.axvline(ELECTRICITY_PRICE_GBP_MWH, color='grey', ls=':', lw=1.3)
ax.set_xlabel('Electricity Price (£/MWh)')
ax.set_ylabel('NPV (£M)')
ax.set_title(f'NPV of Infrastructure Investment  (20yr, {DISCOUNT_RATE*100:.0f}% WACC)')
ax.legend(fontsize=9, title='Carbon price')
ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('£%.0fM'))
ax.set_xlim(20, 120)
plt.tight_layout()
plt.show()

# IRR chart
fig, ax2 = plt.subplots(figsize=(9, 5.5))
for cp, col in zip(carbon_scenarios[:4], colours_npv[:4]):
    irrs = []
    for ep in elec_range:
        r = calculate_npv_irr(ep, cp)
        irrs.append(r.irr_pct if r.irr_pct is not None else np.nan)
    ax2.plot(elec_range, irrs, lw=2.2, color=col, label=f'Carbon = £{cp}/t CO₂e')

ax2.axhline(DISCOUNT_RATE * 100, color='black', ls='--', lw=1.5,
            label=f'WACC = {DISCOUNT_RATE*100:.0f}% (hurdle rate)')
ax2.axhline(0, color='black', lw=0.8)
ax2.axvline(ELECTRICITY_PRICE_GBP_MWH, color='grey', ls=':', lw=1.3)
ax2.set_xlabel('Electricity Price (£/MWh)')
ax2.set_ylabel('IRR (%)')
ax2.set_title('Internal Rate of Return vs. Electricity Price')
ax2.legend(fontsize=9)
ax2.set_xlim(20, 120)
plt.tight_layout()
plt.show()

# Key results at baseline
fin = calculate_npv_irr()
print(f"\nBaseline (£{ELECTRICITY_PRICE_GBP_MWH}/MWh electricity, £{CARBON_PRICE_BASELINE}/t carbon):")
print(f"  NPV:            £{fin.npv_gbp/1e6:.2f}M")
print(f"  IRR:            {fin.irr_pct:.1f}%")
print(f"  Simple payback: {fin.simple_payback_yr:.1f} years")
print(f"  BCR:            {fin.benefit_cost_ratio:.2f}")

---
## 9. NPV Sensitivity Heatmap

In [ ]:
from matplotlib.colors import TwoSlopeNorm

elec_grid   = np.linspace(20, 120, 40)
carbon_grid = np.linspace(20, 200, 40)

Z = np.zeros((len(carbon_grid), len(elec_grid)))
for i, cp in enumerate(carbon_grid):
    for j, ep in enumerate(elec_grid):
        Z[i, j] = calculate_npv_irr(ep, cp).npv_gbp / 1e6

fig, ax = plt.subplots(figsize=(9, 6))
norm = TwoSlopeNorm(vmin=Z.min(), vcenter=0, vmax=Z.max())
im = ax.contourf(elec_grid, carbon_grid, Z, levels=25, cmap='RdYlGn', norm=norm)
ax.contour(elec_grid, carbon_grid, Z, levels=[0], colors='black', linewidths=2.0)
fig.colorbar(im, ax=ax, label='NPV (£M)')

ax.scatter([ELECTRICITY_PRICE_GBP_MWH], [CARBON_PRICE_BASELINE],
           color='white', s=100, zorder=5, edgecolors='black', lw=1.5,
           label=f'Baseline (£{ELECTRICITY_PRICE_GBP_MWH}/MWh, £{CARBON_PRICE_BASELINE}/t)')
ax.set_xlabel('Electricity Price (£/MWh)')
ax.set_ylabel('Carbon Price (£/t CO₂e)')
ax.set_title('NPV Heatmap: Electricity Price × Carbon Price\n(Black contour = NPV breakeven)')
ax.legend(loc='lower right', fontsize=9)
plt.tight_layout()
plt.show()

---
## 10. Tornado Chart – LCOH Parameter Sensitivity

In [ ]:
# Uses explicit override kwargs in calculate_lcoh — avoids the Python
# 'from ... import' local-binding issue that made monkey-patching ineffective.

baseline_lcoh = calculate_lcoh().total_dispensed_cost_gbp_kg

# Map: (display label, kwarg name in calculate_lcoh, baseline value)
params = [
    ('Electricity price',       'electricity_price_gbp_mwh',        ELECTRICITY_PRICE_GBP_MWH),
    ('Electrolyser efficiency', 'electrolyser_efficiency_kwh_kg',    ELECTROLYSER_EFFICIENCY_KWH_KG),
    ('Electrolyser CAPEX',      'electrolyser_cost_per_kw',          ELECTROLYSER_COST_PER_KW),
    ('BoP fraction',            'bop_fraction',                      BOP_FRACTION),
    ('Transport cost',          'transport_cost_gbp_kg',             TRANSPORT_COST_GBP_KG),
    ('HRS OPEX',                'hrs_opex_gbp_kg_override',          HRS_OPEX_GBP_KG),
    ('Discount rate',           'discount_rate',                     DISCOUNT_RATE),
]

results = []
for label, kwarg, base_val in params:
    low  = calculate_lcoh(**{kwarg: base_val * 0.80}).total_dispensed_cost_gbp_kg - baseline_lcoh
    high = calculate_lcoh(**{kwarg: base_val * 1.20}).total_dispensed_cost_gbp_kg - baseline_lcoh
    results.append((label, low, high, abs(high - low)))

results.sort(key=lambda x: x[3], reverse=True)
labels = [r[0] for r in results]
lows   = [r[1] for r in results]
highs  = [r[2] for r in results]

fig, ax = plt.subplots(figsize=(9, 5.5))
y = np.arange(len(labels))
for i, (lo, hi) in enumerate(zip(lows, highs)):
    ax.barh(i, abs(hi - lo), left=min(lo, hi), height=0.5,
            color=PALETTE['red'] if hi > 0 else PALETTE['green'], alpha=0.82)
    ax.text(lo - 0.003, i, f'{lo:+.3f}', va='center', ha='right', fontsize=8.5)
    ax.text(hi + 0.003, i, f'{hi:+.3f}', va='center', ha='left',  fontsize=8.5)

ax.set_yticks(y)
ax.set_yticklabels(labels, fontsize=11)
ax.axvline(0, color='black', lw=1.3)
ax.set_xlabel('Change in Dispensed LCOH (£/kg H₂)')
ax.set_title(f'LCOH Sensitivity to ±20% Parameter Variation\n(Baseline LCOH = £{baseline_lcoh:.2f}/kg)')
from matplotlib.patches import Patch
ax.legend(handles=[
    Patch(color=PALETTE['red'],   alpha=0.82, label='Increases LCOH (+20%)'),
    Patch(color=PALETTE['green'], alpha=0.82, label='Decreases LCOH (+20%)'),
], fontsize=9, loc='lower right')
plt.tight_layout()
plt.show()

print(f"\n{'Parameter':<25} {'Low (−20%)':<18} {'High (+20%)':<18} {'Swing'}")
print('-' * 72)
for label, lo, hi, sw in results:
    print(f"{label:<25} {lo:+.3f} £/kg       {hi:+.3f} £/kg       {sw:.3f} £/kg")

---
## Summary Table

In [ ]:
d   = calculate_demand()
cap = calculate_capex()
lc  = calculate_lcoh(ELECTRICITY_PRICE_GBP_MWH)
ac  = calculate_annual_costs()
em  = calculate_emissions()
fin = calculate_npv_irr()

summary = [
    ('── DEMAND ──',                           ''),
    ('Full fleet daily demand',                f'{d.full_fleet_daily_kg:.0f} kg/day'),
    ('Annual demand',                          f'{d.annual_total_tonnes:.0f} t/yr'),
    ('── CAPEX ──',                            ''),
    ('Total infrastructure CAPEX',             f'£{cap.total_capex_gbp/1e6:.2f}M'),
    ('── LCOH ──',                             ''),
    ('Total dispensed LCOH',                   f'£{lc.total_dispensed_cost_gbp_kg:.2f}/kg'),
    ('Annual H₂ fuel cost',                   f'£{ac.h2_fuel_cost_gbp/1e6:.2f}M'),
    ('Annual diesel equivalent',              f'£{ac.diesel_fuel_cost_gbp/1e6:.2f}M'),
    ('── EMISSIONS ──',                        ''),
    ('Annual CO₂ saving',                     f'{em.co2_saving_tonnes_yr:,.0f} t/yr  ({em.co2_reduction_pct:.0f}% reduction)'),
    ('── FINANCIALS ──',                       ''),
    ('NPV  (20yr, 8% WACC)',                  f'£{fin.npv_gbp/1e6:.2f}M'),
    ('IRR',                                    f"{fin.irr_pct:.1f}%"),
    ('Simple payback',                         f'{fin.simple_payback_yr:.1f} years'),
    ('Benefit-cost ratio',                     f'{fin.benefit_cost_ratio:.2f}'),
]

print(f"\n{'Birmingham H₂ Bus Fleet – Key Results':^55}")
print('=' * 55)
for k, v in summary:
    if v == '':
        print(f"\n  {k}")
    else:
        print(f"  {k:<35} {v}")
print('=' * 55)